#Extract
Authors: Gustavo FLEURY && Induraj RAMAMURTHY

Project: https://github.com/gustavofleury/Audit_Reports_NLP

#Validate the Rows Collected
Test if new columns match with some rules. 





In [0]:
#Libraries
import pandas as pd
import numpy as np
import unicodedata as ud
import re

from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [0]:
#Take Files From GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Take DF files From Drive
fileDFReportInColumns='/content/drive/My Drive/Research/DataSets/DF_ReportsInColumns_AAG_v19.10.24.pkl'
# fileDFReportInColumns='/content/drive/My Drive/Research/DataSets/DF_ReportsInColumns_AAG.pkl'
dfReports=pd.read_pickle(fileDFReportInColumns)

fileName = "/content/drive/My Drive/Research/DataSets/DF_Links_Reports_AAG.pkl"
dfFile=pd.read_pickle(fileName)

#DF of list of OrgSub (From Expenses DataBase)
dfOrgSub=pd.read_pickle("/content/drive/My Drive/Research/DataSets/OrgaoSubordinado_Unique.pkl")

In [0]:
#Verify DFs
print("Number Reports ACC original       : " + str( len(dfFile) ) )
print("Number Reports ACC on DF parsed   : " + str( len(dfReports.FileNumber.unique()) ) )

table = pd.pivot_table(dfReports, values='ConstNumber', index=['FileNumber'], columns=[], aggfunc= ( lambda x: len(x.unique()) ) )
print("Number of Findings per Report- MAX: " + str( max(table.ConstNumber) ) + " MEAN: " + str( np.mean(table.ConstNumber) ) + " MIN: " + str( min(table.ConstNumber) )  )
print("Total of Finds: " + str(len(dfReports)))

table = pd.pivot_table(dfReports, values='ConstNumber', index=['ConstTYPE'], columns=[], aggfunc= ( lambda x: len(x) ))
print("TYPES of Findings: ")
print(table)
print("Total: " + str(sum(table.ConstNumber)))

Number Reports ACC original       : 847
Number Reports ACC on DF parsed   : 842
Number of Findings per Report- MAX: 33 MEAN: 6.324228028503563 MIN: 1
Total of Finds: 5437
TYPES of Findings: 
                             ConstNumber
ConstTYPE                               
                                     317
ERROR-LastConst                       53
ERROR-NoConst                         85
ERROR-Rel_Aval-LastConst               2
ERROR-Rel_Aval-beetwenConst            7
conclusao                            731
constatacao                         2043
informacao                          2199
Total: 5437


In [0]:
# Count Filtered >2015 and the listed Companies
dfReports['Year']=dfReports["DateEnd"].str[0:4]
dfReports.loc[ dfReports['Year'] == 'ERRO', 'Year' ]=0
dfReports['Year']=dfReports['Year'].astype(int, errors='ignore')
dfReports['Year'].unique()
dfReports2015 =  dfReports[ dfReports['Year'] >= 2015 ]
print("Total Findings Year > 2015: " + str(len(dfReports2015 )))
table = pd.pivot_table(dfReports2015, values='ConstNumber', index=['ConstTYPE'], columns=[], aggfunc= ( lambda x: len(x) ))
print("TYPES of Findings: ")
print(table)

Total Findings Year > 2015: 2754
TYPES of Findings: 
                             ConstNumber
ConstTYPE                               
                                     196
ERROR-LastConst                        4
ERROR-NoConst                         29
ERROR-Rel_Aval-LastConst               2
ERROR-Rel_Aval-beetwenConst            7
conclusao                            340
constatacao                         1066
informacao                          1110


In [0]:
# VERIFY THE ERRORS in Filtered >2015
dfReports[ dfReports['ConstTYPE'].str.contains('ERROR') ].FileNumber.to_list()

In [0]:
# Filtered >2015 and in list of OrgSub
dfOrgSub['OrgSub']=dfOrgSub['Nome Órgão Subordinado'].str.decode("utf8")
dfReports['OrgSub']=dfReports['OrgSub'].str.strip()
dfOrgSub['OrgSub']=dfOrgSub['OrgSub'].str.strip()
dfReports2015[ dfReports2015['OrgSub'].isin( dfOrgSub['OrgSub'].tolist() ) ]

,OrgSubCod,OrgSub,FileNumber,ReportNumber,ReportType,DateBegin,DateEnd,ConstNumber,ConstTYPE,ConstTitle,ConstText,Year
35,25901,fundo de compensacao de variacoes salariais,13284,201801708,Auditoria de Acompanhamento da Gestão,2018/1,2018/12,2.1,,inconformidades que nao foram consideradas dis...,inconformidades que nao foram consideradas dis...,2018
36,25901,fundo de compensacao de variacoes salariais,13284,201801708,Auditoria de Acompanhamento da Gestão,2018/1,2018/12,2.2,,distorcoes individualmente relevantes nas demo...,distorcoes individualmente relevantes nas demo...,2018
37,25901,fundo de compensacao de variacoes salariais,13284,201801708,Auditoria de Acompanhamento da Gestão,2018/1,2018/12,2.3,,inconformidades nas demonstracoes contabeis de...,excesso de ajuste para perdas referentes a cre...,2018
89,26254,universidade federal do triangulo mineiro,13374,201411579,Auditoria de Acompanhamento da Gestão,2015/1,2015/12,1.1.1.1,constatacao,"falta de inclusao, no paint/2015, dos criteri...",a unidade de auditoria interna da universidad...,2015
90,26254,universidade federal do triangulo mineiro,13374,201411579,Auditoria de Acompanhamento da Gestão,2015/1,2015/12,1.1.1.2,informacao,das acoes de auditoria objetivando avaliar a ...,objetivando avaliar a situacao atual dos trab...,2015
...,...,...,...,...,...,...,...,...,...,...,...,...
5362,22211,companhia nacional de abastecimento,8934,201600102,Auditoria de Acompanhamento da Gestão,2016/1,2016/5,1.1.1.8,constatacao,falha na documentacao do sistema.,o gestor encaminhou toda documentacao disponi...,2016
5363,22211,companhia nacional de abastecimento,8934,201600102,Auditoria de Acompanhamento da Gestão,2016/1,2016/5,1.1.1.9,constatacao,falta de estrutura para suporte aos usuarios ...,"em reuniao realizada em 11/03/2016, em relaca...",2016
5364,22211,companhia nacional de abastecimento,8934,201600102,Auditoria de Acompanhamento da Gestão,2016/1,2016/5,1.1.1.10,constatacao,inconsistencia na validacao de dados de produ...,em analise ao sistema sican no ambiente de ho...,2016
5365,22211,companhia nacional de abastecimento,8934,201600102,Auditoria de Acompanhamento da Gestão,2016/1,2016/5,1.1.1.11,informacao,protelacao na definicao da area gestora do si...,em analise ao documento de visao de projeto d...,2016


In [0]:
# dfOrgSub.head(3)
# dfOrgSub['Nome Órgão Subordinado'].tolist()[1:3]
# dfReports['OrgSub']=dfReports['OrgSub'].str.strip()
dfOrgSub['OrgSub']=dfOrgSub['OrgSub'].str.strip()
dfOrgSub [ dfOrgSub['OrgSub'].str.contains('universidade federal do parana') ]

,Código Órgão Subordinado,Nome Órgão Subordinado,0,OrgSub
85,26241,b'universidade federal do parana',8187,universidade federal do parana


In [0]:
# OrgSup Unique in Reports
# dfOrgSub=dfOrgSub.drop(columns='OrgSubRep')
dfOrgSub['OrgSubRep']=""
lstOrgRep = dfReports2015["OrgSub"].unique()

for index, row in dfOrgSub.iterrows():
  for rowRep in lstOrgRep:
    if len(rowRep) > 2:
      if similar( row['OrgSub'], rowRep ) >= 0.9 :
        print (str(similar( row['OrgSub'], rowRep )) + "- " + row['OrgSub'] + " - " + rowRep)

      # if rowRep.find( row['OrgSub'] ) > -1 :
      #   # print (row['OrgSub'] + " " + rowRep)
      #   dfOrgSub['OrgSubRep'][   dfOrgSub['OrgSub']==row['OrgSub'] ]=rowRep
      # if row['OrgSub'].find( rowRep ) > -1:
      #   # print (row['OrgSub'] + " " + rowRep)
      #   dfOrgSub['OrgSubRep'][   dfOrgSub['OrgSub']==row['OrgSub'] ]=rowRep

print( len(dfOrgSub['OrgSubRep'].unique()))


0.9- comissao nacional de energia nuclear - comissao nacional de energia nuclear - sede 
0.935064935064935- comissao nacional de energia nuclear - comissao nacional de energia nuclear-ird 
0.9230769230769231- comissao nacional de energia nuclear - comissao nacional de energia nuclear-sede 
0.9428571428571428- nuclebras equipamentos pesados s.a. - nuclebras equipamentos pesados s/a 
0.9855072463768116- nuclebras equipamentos pesados s/a - nuclebras equipamentos pesados s/a 
0.9066666666666666- industrias nucleares do brasil s/a - industrias nucleares do brasil s/a - inb 
0.9855072463768116- financiadora de estudos e projetos - financiadora de estudos e projetos 
0.9787234042553191- departamento nacional de obras contra as secas - departamento nacional de obras contras as secas 
0.9859154929577465- companhia nacional de abastecimento - companhia nacional de abastecimento 
0.9787234042553191- casa da moeda do brasil - casa da moeda do brasil 
0.9056603773584906- casa da moeda do brasil - 

In [0]:
# dfOrgSub=dfOrgSub.drop(columns='OrgSubRep')
# len(dfOrgSub)
# len(dfOrgSub[ dfOrgSub['OrgSubRep']!=""  ] )
# !pip install distance
# import distance
distance.levenshtein("instituto nac. da propriedade industrial", "instituto nacional da propriedade industrial ")
fuzz.token_set_ratio("instituto nac. da propriedade industrial", "instituto nacional da propriedade industrial ")
fuzz.token_set_ratio("depto. nac. de infra-estrutura de transportes", "depart. nac. de infra-estrutura de transporte ") 

95

In [0]:
# similar("Apple","Appel")

# !pip install fuzzywuzzy
# from fuzzywuzzy import fuzz

fuzz.token_set_ratio('Room, 2 Double Beds (19th to 25th Floors)', 'Two Double Beds - Location Room (19th to 25th Floors)')

97

In [0]:
# Create a relation Table between names of OrgSub
dfOrgSub2 = pd.merge( dfOrgSub, dfReports2015[ ['OrgSub','FileNumber' ]], on='OrgSub', how='left')

In [0]:
# dfOrgSub2[ ~dfOrgSub2['FileNumber'].isna() ]
#  ~dfOrgSub2['FileNumber'].isna()
# dfOrgSub2['FileNumber'].
lstOrgRep = dfReports2015["OrgSub"].unique()
len(lstOrgRep)

150

In [0]:
#TEST - Not possible to use Publication Data, because it all recent;
#Create YearPub
dfFile['YearPub']= dfFile['Publicado em'].str[-4:].astype(int)
dfFile.head(2)

#Filter YearPub >= 2015
dfFileFilter = dfFile[ dfFile["YearPub"]>=2015 ]
print("Original: " + str(len(dfFile)) + " Filtered: " + str(len(dfFileFilter)))

dfFile["YearPub"].value_counts()